In [1]:
from neo4j import GraphDatabase

In [2]:
uri = "bolt://localhost:7687"
username = "neo4j"
password = "12345678"
auth=(username, password)
driver = GraphDatabase.driver(uri, auth=(username, password))

In [3]:
node_id = "Disease::DOID:184"

In [9]:
def query1(driver, node_id):
    query = """
        MATCH (n {id: $node_id})
        OPTIONAL MATCH (n)-[r1]->(target)
        WHERE r1.metaedge IN ['DlA', 'DaG']
        OPTIONAL MATCH (source)-[r2]->(n)
        WHERE r2.metaedge IN ['CpD', 'CtD']
        RETURN n, COLLECT(DISTINCT target) AS target_nodes, COLLECT(DISTINCT source) AS source_nodes
    """
    with driver.session() as session:
        result = session.run(query, node_id=node_id)
        return result.data()  

In [10]:
result = query1(driver, node_id)

In [11]:
result

[{'n': {'kind': 'Disease', 'name': 'bone cancer', 'id': 'Disease::DOID:184'},
  'target_nodes': [{'kind': 'Anatomy',
    'name': 'nose',
    'id': 'Anatomy::UBERON:0000004'},
   {'kind': 'Anatomy', 'name': 'appendage', 'id': 'Anatomy::UBERON:0000026'},
   {'kind': 'Anatomy', 'name': 'tendon', 'id': 'Anatomy::UBERON:0000043'},
   {'kind': 'Anatomy',
    'name': 'cranial nerve II',
    'id': 'Anatomy::UBERON:0000941'},
   {'kind': 'Anatomy', 'name': 'sternum', 'id': 'Anatomy::UBERON:0000975'},
   {'kind': 'Anatomy', 'name': 'leg', 'id': 'Anatomy::UBERON:0000978'},
   {'kind': 'Anatomy',
    'name': 'vertebral arch joint',
    'id': 'Anatomy::UBERON:0001067'},
   {'kind': 'Anatomy',
    'name': 'external carotid artery',
    'id': 'Anatomy::UBERON:0001070'},
   {'kind': 'Anatomy',
    'name': 'vertebral bone 2',
    'id': 'Anatomy::UBERON:0001093'},
   {'kind': 'Anatomy',
    'name': 'clavicle bone',
    'id': 'Anatomy::UBERON:0001105'},
   {'kind': 'Anatomy',
    'name': 'vertebral colum

In [20]:
def query2(driver):
    query = """
    MATCH (disease)
    WHERE disease.kind = 'Disease'
    AND NOT EXISTS {
        MATCH (disease)-[]-(compound)
        WHERE compound.kind = 'Compound'
    }
    WITH COLLECT(disease) AS diseases
    MATCH (disease)-[disAnatRel]->(anatomy)
    WHERE disease IN diseases
      AND disAnatRel.metaedge = 'DlA' 
      AND anatomy.kind = 'Anatomy'
    WITH COLLECT(DISTINCT anatomy) AS anatomies
    MATCH (compound)-[compoundGeneRel]->(gene)<-[anatomyGeneRel]-(anatomy)
    WHERE compound.kind = 'Compound'
      AND gene.kind = 'Gene'
      AND anatomy IN anatomies
      AND (
          (compoundGeneRel.metaedge = 'CuG' AND anatomyGeneRel.metaedge = 'AdG')
          OR
          (compoundGeneRel.metaedge = 'CdG' AND anatomyGeneRel.metaedge = 'AuG')
      )
    
    RETURN DISTINCT compound
    """
    
    with driver.session() as session:
        result = session.run(query)
        return list(result.data())

In [23]:
compounds = query2(driver)

In [22]:
len(compounds)

783

In [24]:
compounds

[{'compound': {'kind': 'Compound',
   'name': 'Quinacrine',
   'id': 'Compound::DB01103'}},
 {'compound': {'kind': 'Compound',
   'name': 'Topotecan',
   'id': 'Compound::DB01030'}},
 {'compound': {'kind': 'Compound',
   'name': 'Mitoxantrone',
   'id': 'Compound::DB01204'}},
 {'compound': {'kind': 'Compound',
   'name': 'Vorinostat',
   'id': 'Compound::DB02546'}},
 {'compound': {'kind': 'Compound',
   'name': 'Lovastatin',
   'id': 'Compound::DB00227'}},
 {'compound': {'kind': 'Compound',
   'name': 'Mitomycin',
   'id': 'Compound::DB00305'}},
 {'compound': {'kind': 'Compound',
   'name': 'Vindesine',
   'id': 'Compound::DB00309'}},
 {'compound': {'kind': 'Compound',
   'name': 'Vinorelbine',
   'id': 'Compound::DB00361'}},
 {'compound': {'kind': 'Compound',
   'name': 'Treprostinil',
   'id': 'Compound::DB00374'}},
 {'compound': {'kind': 'Compound',
   'name': 'Amlodipine',
   'id': 'Compound::DB00381'}},
 {'compound': {'kind': 'Compound',
   'name': 'Alprazolam',
   'id': 'Compound